In [1]:
import os
import dtale
import pandas as pd
import numpy as np

df = pd.read_csv(os.path.expanduser("~/Documents/data_db-in-documents/gym_exercise_sessions_202507090754.csv"))
# d = dtale.show(df)

In [2]:
movement_by_date = df.groupby("fmt_exercise_date")["fmt_movement"].unique().reset_index()
# movement_by_date.columns = ["日期", "当日训练动作列表"]
movement_by_date

,fmt_exercise_date,fmt_movement
0,2024-11-08,[BAR_HOLDS]
1,2024-11-11,"[SHOULDER_PRESS, BARBELL_ROW, WIDE_GRIP_LAT_PU..."
2,2024-11-12,"[FLYE, UNKNOWN, CHEST_PRESS_WITH_BAND]"
3,2024-11-13,"[LEG_ADDUCTION, WEIGHTED_LEG_EXTENSIONS, STATI..."
4,2024-11-14,"[UNKNOWN, CURL, WEIGHTED_LEG_CURL, FLYE, CHEST..."
...,...,...
119,2025-07-01,"[ONE_ARM_CABLE_LATERAL_RAISE, CALF_RAISE, LEG_..."
120,2025-07-02,"[CABLE_BICEPS_CURL, CABLE_OVERHEAD_TRICEPS_EXT..."
121,2025-07-04,"[DUMBBELL_BICEPS_CURL, CHEST_PRESS_WITH_BAND, ..."
122,2025-07-07,"[INCLINE_DUMBBELL_BICEPS_CURL, DUMBBELL_BICEPS..."


In [3]:
grouped = df.groupby("fmt_exercise_date")[["fmt_movement", "duration", "weight", "repetition_count"]]
print(grouped)

In [4]:
# 先处理空 weight
df["adjusted_weight"] = df["weight"].fillna(98000 / 2)

# 判断是否为 FARMERS_WALK，构造容量列
df["volume_kg"] = np.where(
    df["fmt_movement"].str.upper() == "FARMERS_WALK",
    df["duration"] * df["repetition_count"] * df["adjusted_weight"] / 1000,
    df["repetition_count"] * df["adjusted_weight"] / 1000
)

# 按日期 + 动作名称分组统计容量总和
daily_volume = (
    df.groupby(["fmt_exercise_date", "fmt_movement"])["volume_kg"]
    .sum()
    .reset_index()
    .rename(columns={
        "fmt_exercise_date": "日期",
        "fmt_movement": "动作",
        "volume_kg": "总容量（kg）"
    })
    .sort_values(["日期", "动作"])
)

df

,id,activity_id,category,name,duration,repetition_count,weight,set_type,start_time,fmt_start_time,fmt_exercise_date,fmt_movement,created_at,updated_at,adjusted_weight,volume_kg
0,4231,494389971,CURL,DUMBBELL_BICEPS_CURL,37.026,8,12000.0,ACTIVE,2025-07-04 10:38:56.000,2025-07-04 18:38:56,2025-07-04,DUMBBELL_BICEPS_CURL,2025-07-08 07:24:20.567,2025-07-09 07:17:58.692,12000.0,96.0
1,4232,494389971,CURL,DUMBBELL_BICEPS_CURL,30.344,6,14000.0,ACTIVE,2025-07-04 10:40:46.000,2025-07-04 18:40:46,2025-07-04,DUMBBELL_BICEPS_CURL,2025-07-08 07:24:20.579,2025-07-09 07:17:58.712,14000.0,84.0
2,4233,494388624,PUSH_UP,CHEST_PRESS_WITH_BAND,32.803,10,42000.0,ACTIVE,2025-07-04 10:33:46.000,2025-07-04 18:33:46,2025-07-04,CHEST_PRESS_WITH_BAND,2025-07-08 07:24:20.593,2025-07-09 07:17:58.727,42000.0,420.0
3,4234,494388624,PUSH_UP,CHEST_PRESS_WITH_BAND,31.569,10,42000.0,ACTIVE,2025-07-04 10:34:40.000,2025-07-04 18:34:40,2025-07-04,CHEST_PRESS_WITH_BAND,2025-07-08 07:24:20.602,2025-07-09 07:17:58.745,42000.0,420.0
4,4235,494388624,PUSH_UP,CHEST_PRESS_WITH_BAND,30.751,12,42000.0,ACTIVE,2025-07-04 10:35:35.000,2025-07-04 18:35:35,2025-07-04,CHEST_PRESS_WITH_BAND,2025-07-08 07:24:20.611,2025-07-09 07:17:58.761,42000.0,504.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2095,5616,443950743,FLYE,INCLINE_REVERSE_FLYE,45.455,8,49000.0,ACTIVE,2025-01-06 01:46:11.000,2025-01-06 09:46:11,2025-01-06,INCLINE_REVERSE_FLYE,2025-07-08 07:24:42.467,2025-07-09 07:18:24.293,49000.0,392.0
2096,5720,442394642,SHOULDER_PRESS,NaN,29.681,12,28000.0,ACTIVE,2024-12-30 13:00:43.000,2024-12-30 21:00:43,2024-12-30,SHOULDER_PRESS,2025-07-08 07:24:44.410,2025-07-09 07:18:25.647,28000.0,336.0
2097,4681,481678870,CALF_RAISE,NaN,40.991,20,54500.0,ACTIVE,2025-05-27 01:15:31.000,2025-05-27 09:15:31,2025-05-27,CALF_RAISE,2025-07-08 07:24:26.875,2025-07-09 07:18:06.498,54500.0,1090.0
2098,4720,481133195,BENCH_PRESS,INCLINE_DUMBBELL_BENCH_PRESS,48.791,8,14000.0,ACTIVE,2025-05-25 09:48:54.000,2025-05-25 17:48:54,2025-05-25,INCLINE_DUMBBELL_BENCH_PRESS,2025-07-08 07:24:27.523,2025-07-09 07:18:07.093,14000.0,112.0


In [5]:
movement_to_bodypart = {
    "SIT_UP": "ABS",
    "CRUNCH": "ABS",
    "LEG_PRESS": "LEG",
    "FARMERS_WALK": "GRIP",
    "DUMBBELL_BICEPS_CURL": "ARM",
    "CHEST_PRESS_WITH_BAND": "CHEST",
    "BARBELL_ROW": "BACK",
    "SHOULDER_PRESS": "SHOULDER",
    # 你可以继续添加...
}

# 1. 替换缺失 weight
df["adjusted_weight"] = df["weight"].fillna(98000 / 2)

# 2. 计算 volume（容量）
df["volume_kg"] = np.where(
    df["fmt_movement"].str.upper() == "FARMERS_WALK",
    df["duration"] * df["repetition_count"] * df["adjusted_weight"] / 1000,
    df["repetition_count"] * df["adjusted_weight"] / 1000
)

# 3. 标准化动作名为大写用于匹配映射
df["movement_upper"] = df["fmt_movement"].str.upper()

# 4. 准备动作到身体部位的映射
movement_to_bodypart = {
    "SIT_UP": "ABS",
    "CRUNCH": "ABS",
    "LEG_PRESS": "LEG",
    "FARMERS_WALK": "GRIP",
    "DUMBBELL_BICEPS_CURL": "ARM",
    "CHEST_PRESS_WITH_BAND": "CHEST",
    "BARBELL_ROW": "BACK",
    "SHOULDER_PRESS": "SHOULDER",
    # 更多动作...
}

# 5. 映射到新列
df["body_part"] = df["movement_upper"].map(movement_to_bodypart).fillna("OTHER")

# 6. 按天统计每个身体部位的总容量
daily_body_volume = (
    df.groupby(["fmt_exercise_date", "body_part"])["volume_kg"]
    .sum()
    .reset_index()
    .rename(columns={
        "fmt_exercise_date": "日期",
        "body_part": "身体部位",
        "volume_kg": "总容量（kg）"
    })
    .sort_values(["日期", "身体部位"])
)

df

,id,activity_id,category,name,duration,repetition_count,weight,set_type,start_time,fmt_start_time,fmt_exercise_date,fmt_movement,created_at,updated_at,adjusted_weight,volume_kg,movement_upper,body_part
0,4231,494389971,CURL,DUMBBELL_BICEPS_CURL,37.026,8,12000.0,ACTIVE,2025-07-04 10:38:56.000,2025-07-04 18:38:56,2025-07-04,DUMBBELL_BICEPS_CURL,2025-07-08 07:24:20.567,2025-07-09 07:17:58.692,12000.0,96.0,DUMBBELL_BICEPS_CURL,ARM
1,4232,494389971,CURL,DUMBBELL_BICEPS_CURL,30.344,6,14000.0,ACTIVE,2025-07-04 10:40:46.000,2025-07-04 18:40:46,2025-07-04,DUMBBELL_BICEPS_CURL,2025-07-08 07:24:20.579,2025-07-09 07:17:58.712,14000.0,84.0,DUMBBELL_BICEPS_CURL,ARM
2,4233,494388624,PUSH_UP,CHEST_PRESS_WITH_BAND,32.803,10,42000.0,ACTIVE,2025-07-04 10:33:46.000,2025-07-04 18:33:46,2025-07-04,CHEST_PRESS_WITH_BAND,2025-07-08 07:24:20.593,2025-07-09 07:17:58.727,42000.0,420.0,CHEST_PRESS_WITH_BAND,CHEST
3,4234,494388624,PUSH_UP,CHEST_PRESS_WITH_BAND,31.569,10,42000.0,ACTIVE,2025-07-04 10:34:40.000,2025-07-04 18:34:40,2025-07-04,CHEST_PRESS_WITH_BAND,2025-07-08 07:24:20.602,2025-07-09 07:17:58.745,42000.0,420.0,CHEST_PRESS_WITH_BAND,CHEST
4,4235,494388624,PUSH_UP,CHEST_PRESS_WITH_BAND,30.751,12,42000.0,ACTIVE,2025-07-04 10:35:35.000,2025-07-04 18:35:35,2025-07-04,CHEST_PRESS_WITH_BAND,2025-07-08 07:24:20.611,2025-07-09 07:17:58.761,42000.0,504.0,CHEST_PRESS_WITH_BAND,CHEST
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2095,5616,443950743,FLYE,INCLINE_REVERSE_FLYE,45.455,8,49000.0,ACTIVE,2025-01-06 01:46:11.000,2025-01-06 09:46:11,2025-01-06,INCLINE_REVERSE_FLYE,2025-07-08 07:24:42.467,2025-07-09 07:18:24.293,49000.0,392.0,INCLINE_REVERSE_FLYE,OTHER
2096,5720,442394642,SHOULDER_PRESS,NaN,29.681,12,28000.0,ACTIVE,2024-12-30 13:00:43.000,2024-12-30 21:00:43,2024-12-30,SHOULDER_PRESS,2025-07-08 07:24:44.410,2025-07-09 07:18:25.647,28000.0,336.0,SHOULDER_PRESS,SHOULDER
2097,4681,481678870,CALF_RAISE,NaN,40.991,20,54500.0,ACTIVE,2025-05-27 01:15:31.000,2025-05-27 09:15:31,2025-05-27,CALF_RAISE,2025-07-08 07:24:26.875,2025-07-09 07:18:06.498,54500.0,1090.0,CALF_RAISE,OTHER
2098,4720,481133195,BENCH_PRESS,INCLINE_DUMBBELL_BENCH_PRESS,48.791,8,14000.0,ACTIVE,2025-05-25 09:48:54.000,2025-05-25 17:48:54,2025-05-25,INCLINE_DUMBBELL_BENCH_PRESS,2025-07-08 07:24:27.523,2025-07-09 07:18:07.093,14000.0,112.0,INCLINE_DUMBBELL_BENCH_PRESS,OTHER
